In [ ]:
# PART 1 (ProcessDicom)
# dataset_dir = "/home/rodrigo/processed-dataset/"
# zip_list = glob(dataset_dir + "*.zip")
# extract_dicom_zips(zip_list)

# PART 2 (ChooseBestScans)
# root_dir = '/home/rodrigo/processed-dataset/'
# exam_dirs = get_exam_dirs(root_dir)
# exams_dictionary = gen_exams_dictionary(exam_dirs)
# best_scans_dict = check_best_scan(exams_dictionary)
# save_best_scans(root_dir, best_scans_dict)

# PART 3 (ProcessDicom)
# gen_all_pngs(root_dir)

# PART 4 (ProcessImages)
# dataset_dir = "/home/rodrigo/processed-dataset/"
# dir_list = glob(dataset_dir + "[0-9]*")
# total = len(dir_list)
#
# complete = 1
# for exam_dir in dir_list:
#     print("processing exam " + os.path.basename(exam_dir))
#     image_dir = exam_dir + "/pngs"
#     filelist = os.listdir(image_dir)
#     process_exam(filelist, image_dir)
#     print(str(complete) + "/" + str(total))
#     complete += 1

# PART 5 (PrepareTrainingData)
# root_dir = "/home/rodrigo/processed-dataset/"
# chosen_scans = root_dir + "GoodDirs.txt"
# labels_file = root_dir + "labels-batch.csv"
#
# serie_list = read_serie_paths(chosen_scans)
# labels = read_labels(labels_file)
# save_slices_labels(serie_list, labels)


# PART 6 (Model)

#### This will be used to extract, structure folders and generate pngs for dicoms in dataset CQ500

# Import Required Libraries

In [2]:
import os
import re
import gdcm
import zipfile
import operator
from glob import glob
from shutil import rmtree
from pathlib import Path
import subprocess

# Extract and Move Dicom files

In [ ]:
def create_folder_structure(dataset_dir, exam_number):
    exam_dir = dataset_dir + exam_number
    dcm_dir = exam_dir + "/dicoms/"; png_dir = exam_dir + "/pngs/"
    proc_dir = exam_dir + "/processed/"; hist_dir = exam_dir + "/histograms/"
    os.makedirs(dcm_dir, exist_ok=True); os.makedirs(png_dir, exist_ok=True);
    os.makedirs(proc_dir, exist_ok=True); os.makedirs(hist_dir, exist_ok=True)
    return dcm_dir

def extract_dicom_zips(zip_list):
    count = 0
    for zip_path in zip_list:
        exam_number = re.findall(r'\d+', zip_path)[-1]
        dcm_dir = create_folder_structure(dataset_dir, exam_number)
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(dcm_dir)
            count += 1
            print(str(count) + "/" + str(len(zip_list)))
        
dataset_dir = "/home/rodrigo/processed-dataset/"
zip_list = glob(dataset_dir + "*.zip")
extract_dicom_zips(zip_list)

1/491
2/491
3/491
4/491
5/491
6/491
7/491
8/491
9/491
10/491
11/491
12/491
13/491
14/491
15/491
16/491
17/491
18/491
19/491
20/491
21/491
22/491
23/491
24/491
25/491
26/491
27/491
28/491
29/491
30/491
31/491
32/491
33/491
34/491
35/491
36/491
37/491
38/491
39/491
40/491
41/491
42/491
43/491
44/491
45/491
46/491
47/491
48/491
49/491
50/491
51/491
52/491
53/491
54/491
55/491
56/491
57/491
58/491
59/491
60/491
61/491
62/491
63/491
64/491
65/491
66/491
67/491
68/491
69/491
70/491
71/491
72/491
73/491
74/491
75/491
76/491
77/491
78/491
79/491
80/491
81/491
82/491
83/491
84/491
85/491
86/491
87/491
88/491
89/491
90/491
91/491
92/491
93/491
94/491
95/491
96/491
97/491
98/491
99/491
100/491
101/491
102/491
103/491
104/491
105/491
106/491
107/491
108/491
109/491
110/491
111/491
112/491
113/491
114/491
115/491
116/491
117/491
118/491
119/491
120/491
121/491
122/491
123/491
124/491
125/491
126/491
127/491
128/491
129/491
130/491
131/491
132/491
133/491
134/491
135/491
136/491
137/491
138/491
139/

# Order and Rename Serie by Z coordinate

In [5]:
def order_all_series(scans_file):
    f = open(scans_file, "r")
    for serie_path in f.readlines():
        serie_path = serie_path.strip("\n")
        sorted_serie = order_serie(serie_path)
        rename_serie_instances(serie_path, sorted_serie)
        
def order_serie(serie_dir):
    dcm_z = {}
    print(serie_dir)
    for dcm in glob(serie_dir + "/*.dcm"):
        result = os.popen("gdcminfo " + dcm).read().splitlines()[-5]
        Z = result.split(",")[-1].strip(")")
        dcm_z[os.path.basename(dcm)] = float(Z)
    sorted_by_z = sorted(dcm_z.items(), key=operator.itemgetter(1))
    return sorted_by_z

def rename_serie_instances(serie_path, sorted_by_z):    
    count = 0
    for dcm in sorted_by_z:
        os.rename(serie_path + "/" + dcm[0], serie_path + "/CT" + str(count) + ".dcm")
        count = count + 1
             
# root_dir = '/home/rodrigo/Batch/'
# good_scans = root_dir + 'GoodDirs.txt'
# order_all_series(good_scans)

/home/rodrigo/Batch/0/dicoms/CT_Plain
/home/rodrigo/Batch/102/dicoms/CT_Plain
/home/rodrigo/Batch/109/dicoms/CT_5mm
/home/rodrigo/Batch/159/dicoms/CT_5mm-2
/home/rodrigo/Batch/243/dicoms/CT_Plain
/home/rodrigo/Batch/303/dicoms/CT_PRE_CONTRAST_5MM_STD
/home/rodrigo/Batch/320/dicoms/CT_55mm_Plain
/home/rodrigo/Batch/486/dicoms/CT_Plain
/home/rodrigo/Batch/490/dicoms/CT_Plain
/home/rodrigo/Batch/89/dicoms/CT_PRE_CONTRAST_5MM_STD


# Convert Dicom to PNG (decompress if necessary)
#### its important to order the slices first

In [23]:
# convert compressed dicom to raw format
def convert_to_raw(src, dst):
    subprocess.run(["gdcmconv", "--raw", src, dst])
# convert dicom to png
def dcm_to_png(src, dst):
    subprocess.run(["dcm2pnm", "+on", "+Sxv", "224", "+Wi", "1", src, dst])
    
def generate_pngs(serie_path):
    # /home/rodrigo/Batch/0/dicoms/serie_name
    exam_dir = os.path.dirname(os.path.dirname(serie_path))
    for dcm_path in glob(serie_path + "/*.dcm"):
        compression = os.popen("gdcminfo " + dcm_path).read().splitlines()[1]
        png_name = Path(dcm_path).stem + ".png"
        if "JPEG" in compression:
            raw_path = exam_dir + "/pngs/" + os.path.basename(dcm_path)
            convert_to_raw(dcm_path, raw_path)
            dcm_to_png(raw_path, exam_dir + "/pngs/" + png_name)
            os.remove(raw_path)
        else:
            dcm_to_png(dcm_path, exam_dir + "/pngs/" + png_name)

def gen_all_pngs(root_dir):
    f = open(root_dir + "GoodDirs.txt", "r")
    scans = f.readlines()
    n_scans = len(scans)
    x = 0
    for serie_path in scans:
        serie_path = serie_path.strip("\n")
        if os.path.isdir(serie_path):
            print(serie_path)
            generate_pngs(serie_path)
            print(str(x) + "/" + str(n_scans))
            x += 1
        
root_dir = '/home/rodrigo/Batch/'
gen_all_pngs(root_dir)

/home/rodrigo/Batch/0/dicoms/CT_Plain
0/10
/home/rodrigo/Batch/102/dicoms/CT_Plain
1/10
/home/rodrigo/Batch/109/dicoms/CT_5mm
2/10
/home/rodrigo/Batch/159/dicoms/CT_5mm-2
3/10
/home/rodrigo/Batch/243/dicoms/CT_Plain
4/10
/home/rodrigo/Batch/303/dicoms/CT_PRE_CONTRAST_5MM_STD
5/10
/home/rodrigo/Batch/320/dicoms/CT_55mm_Plain
6/10
/home/rodrigo/Batch/486/dicoms/CT_Plain
7/10
/home/rodrigo/Batch/490/dicoms/CT_Plain
8/10
/home/rodrigo/Batch/89/dicoms/CT_PRE_CONTRAST_5MM_STD
9/10


#### not used anymore

In [ ]:
# def move_dicoms_recursively(exam_dir):
#     dicoms = glob(exam_dir + "/dicoms/**/*.dcm", recursive=True)
#     count2 = 0
#     dir = exam_dir + "/dicoms/"
#     try:
#         subdir = [dI for dI in os.listdir(dir) if os.path.isdir(os.path.join(dir,dI))][0]
#         for filepath in dicoms:
#             dicom_name = os.path.basename(exam_dir) + "-" + str(count2) + ".dcm" 
#             os.rename(filepath, dir + dicom_name)
#             count2 = count2 + 1
#         print(str(count2) + "/" + str(len(dicoms)))
#         rmtree(dir + subdir)
#     except IndexError:
#         print("No dicoms to move.")

# def move_all_dicoms(dir_list):
#     x = 0
#     for exam_dir in dir_list:
#         move_dicoms_recursively(exam_dir)
#         print(str(x) + "/" + str(len(dir_list)))
#         x = x + 1
# dir_list = 
# move_all_dicoms(dir_list)